# **2019 2nd ML month with KaKR**
**House Price Prediction Challenge**
현우님의 커널을 베이스로 다른 커널들을 참고하며 공부중입니다 
    * https://www.kaggle.com/chocozzz/house-price-prediction-eda-updated-2019-03-12/notebook

- **[Updated]**  2019.03.22 - 기존 작성되지 않은 Description 추가 및 Scaling vs. Normalization을 추가했습니다.
- **[Updated]**  2019.03.25 - 예제를 제외한 부분에서 기존 numpy를 통해 작성한 부분을 변경했습니다(공부목적의 이유). 또한, bathrooms 설명 오류 부분을 수정하였습니다.
- **[Updated]**  2019.04.12 - 코드 상세 설명 추가 및 파생변수 부분을 수정하였습니다. Randomforest 내 RandomizedSearch 추가 및 XGBoost 모델을 추가하였습니다. 
- **[Updated]**  2019.04.15 - lgbm 모델을 추가하였습니다. 파생변수 추가 및 그로 인한 코드 수정이 있습니다.

# **목차**
- <a href='#1'>1. Data Fields</a>  
- <a href='#2'>2. Exploratory Data Analysis(EDA)</a>  
    - <a href='#21'>2-1. 결측치 확인</a>
    - <a href='#22'>2-2. 변수별 유니크 값 개수 확인</a>
    - <a href='#23'>2-3. 종속변수(Y)</a>
    - <a href='#231'>2-3-1. Scaling vs. Normalization</a>
    - <a href='#232'>2-3-2. 정규화(Normalization)</a>
    - <a href='#24'>2-4. 독립변수(X)</a>
    - <a href='#241'>2-4-1. 정규화(Normalization)</a>
- <a href='#3'>3. 이상치 처리</a>  
    - <a href='#32'>3-1. 독립변수(X)</a>
- <a href='#4'>4. 파생변수 생성</a>  
- <a href='#5'>5. 모델링</a>  
    - <a href='#51'>5-1. Linear Regression</a>
    - <a href='#52'>5-2. Random Forest</a>
    - <a href='#53'>5-3. XGBoost</a>

--------------------------------------------------------------------------------------------------------------------------

# **<a id='1'> 1. Data Fields </a>**

* ID : 집을 구분하는 번호
* date : 집을 구매한 날짜
* price : 집의 가격(Target variable)
* bedrooms : 침실의 수
* bathrooms : 화장실의 수
* sqft_living : 주거 공간의 평방 피트(면적)
* sqft_lot : 부지의 평방 피트(면적)
* floors : 집의 층 수
* waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
* view : 집이 얼마나 좋아 보이는지의 정도
* condition : 집의 전반적인 상태
* grade : King County grading 시스템 기준으로 매긴 집의 등급
* sqft_above : 지하실을 제외한 평방 피트(면적)
* sqft_basement : 지하실의 평방 피트(면적)
* yr_built : 지어진 년도
* yr_renovated : 집을 재건축한 년도
* zipcode : 우편번호
* lat : 위도
* long : 경도
* sqft_living15 : 2015년 기준 주거 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
* sqft_lot15 : 2015년 기준 부지의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
--------------------------------------------------------------------------------------------------------------------------

가장 먼저, Data fields의 변수들을 보며 데이터를 이해하고,  설명을 보며 파생변수가 될 수 있는 것들을 체크했습니다.
    * bedrooms와 bathrooms는 각 room의 개수를 나타내고 있어 비슷한 특성을 띈다 파악할 수 있습니다, 
      이를 합하여 bedroom과 bathroom을 변수 생성할 수 있을 것입니다
    * sqft 관련 변수들의 경우, 면적의 영향이 있기에 면적이 클 경우 가격이 높을 수 밖에 없는 관계가 
      있으므로, row간 상대적인 비교를 위해서는 면적의 영향을 줄이는게 좋을 듯 합니다. 이를 위한 방안은 추후 생각 해볼 수 있을 것입니다.
    * 모든 면적을 더하고, 이를 집값으로 나누어 면적 당 평균가 변수를 생성할 수 있을 것이라 생각합니다.
    * sqft_living15 - sqft_living을 하여 변화의 차이를 파생변수로 생성 할 수 있을 것이라 생각합니다.

In [ ]:
# Loading packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy import stats
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline
import gc

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
train.head()

head를 통해 간략히 데이터를 볼 때, 다음을 알 수 있었습니다.
  * sqft_living에서 sqft_living15로 가며 면적이 다소 증가함이 보입니다.
  * sqft_lot은 sqft_living과 다르게 sqft_lot15로 가며 다소 감소했음을 알 수 있습니다.

# **<a id='2'> 2. Exploratory Data Analysis(EDA) </a>**

--------------------------------------------------------------------------------------------------------------------------

### **<a id='21'> 2-1. 결측치 확인 </a>**

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

* 본 competition의 데이터는 결측치가 없이 clean하게 제공되었음을 확인하였습니다.

### **<a id='22'> 2-2. 변수별 유니크 값 개수 확인 </a>**

In [ ]:
train.nunique()

  * waterfront는 이진 변수임을 알 수 있습니다.
  * condition, view는 정도를 나타내는 Rank변수로 예상됩니다.

자세한 사항은 앞으로 EDA를 통해 변수들을 자세히 알아보도록 하겠습니다.

### **<a id='23'> 2-3. 종속변수(Y) </a>**

In [ ]:
train['price'].describe()

* min값과 1분위수(first quartile)값의 자리수가 다름을 알 수 있습니다.
* 3분위수(third quantile)값과 max값의 자리수가 다름을 알 수 있습니다. 
* min값과 1분위수, 각 분위수들의 차이보다도 75%(3Q)에서 max값의 차이가 더욱 큼을 알 수 있습니다.

In [ ]:
#histogram
f, ax = plt.subplots(figsize = (8, 6))
sns.distplot(train['price'])

In [ ]:
train['price'].skew()

In [ ]:
train['price'].kurt()

* 왜도(skewness)는 정규분포와 비교하여 왼쪽으로 꼬리가 길수록(left-skewed), 더욱 큰 음수값을 가지며
  정규분포와 비교하여 오른쪽으로 꼬리가 길수록(right-skewed), 양의 값이 더욱 커집니다.
* 첨도(kurtosis)값은 강하게 볼때는 2이내 (k<2), 완화하여 볼 경우 3이내(k<3)까지를 정규분포에 가깝게 평탄하다 해석합니다.

#### 위의 histogram 및 skew, kurt값을 통해 종속변수가 right-skewed 되어있음을 확인하여 skew를 완화 해주고자 합니다.

--------------------------------------------------------------------------------------------------------------------------
** - data의 skew 완화를 설명하기 이전, 다소 헷갈릴 수 있는 Scaling과 Normalization의 정의를 알아보고자 합니다.**

* 해당 정의는 https://www.kaggle.com/jfeng1023/data-cleaning-challenge-scale-and-normalize-data를 참조하였습니다.

--------------------------------------------------------------------------------------------------------------------------

## **<a id='231'> 2-3-1. Scaling vs. Normalization </a>**
One of the reasons that it's easy to get confused between scaling and normalization is because the terms are sometimes used interchangeably and, to make it even more confusing, they are very similar! In both cases, you're transforming the values of numeric variables so that the transformed data points have specific helpful properties. **The difference is that, in scaling, you're changing the range of your data while in normalization you're changing the shape of the distribution of your data**. Let's talk a little more in-depth about each of these options

## **Scaling**
This means that you're transforming your data so that it fits within a specific scale, like 0-100 or 0-1. You want to scale data when you're using methods based on measures of how far apart data points, like support vector machines, or SVM or k-nearest neighbors, or KNN. With these algorithms, a change of "1" in any numeric feature is given the same importance.

For example, you might be looking at the prices of some products in both Yen and US Dollars. One US Dollar is worth about 100 Yen, but if you don't scale your prices methods like SVM or KNN will consider a difference in price of 1 Yen as important as a difference of 1 US Dollar! This clearly doesn't fit with our intuitions of the world. With currency, you can convert between currencies. But what about if you're looking at something like height and weight? It's not entirely clear how many pounds should equal one inch (or how many kilograms should equal one meter).

By scaling your variables, you can help compare different variables on equal footing. To help solidify what scaling looks like, let's look at a made-up example. (Don't worry, we'll work with real data in just a second, this is just to help illustrate my point.)

In [ ]:
# Scaling Example
# generate 1000 data points randomly drawn from an exponential distribution
from mlxtend.preprocessing import minmax_scaling
import numpy
original_data = numpy.random.exponential(size = 1000)

# mix-max scale the data between 0 and 1
scaled_data = minmax_scaling(original_data, columns = [0])

# plot both together to compare
fig, ax=plt.subplots(1,2)
sns.distplot(original_data, ax=ax[0])
ax[0].set_title("Original Data")
sns.distplot(scaled_data, ax=ax[1])
ax[1].set_title("Scaled data")
del numpy #import numpy를 해제

* 예시에서 데이터의 shape 변화 없이 0 to 8이던 범위를 0 to 1로 변화시켜주었습니다.
* 이와 같이, **shape의 변화 없이 range 만을 조정하는 것이 scaling입니다.**

## **Normalization**
Scaling just changes the range of your data. Normalization is a more radical transformation. The point of normalization is to change your observations so that they can be described as a normal distribution.

> **Normal distribution**: Also known as the "bell curve", this is a specific statistical distribution where a roughly equal observations fall above and below the mean, the mean and the median are the same, and there are more observations closer to the mean. The normal distribution is also known as the Gaussian distribution.

In general, you'll only want to normalize your data if you're going to be using a machine learning or statistics technique that assumes your data is normally distributed. Some examples of these include t-tests, ANOVAs, linear regression, linear discriminant analysis (LDA) and Gaussian naive Bayes. (Pro tip: any method with "Gaussian" in the name probably assumes normality.)

The method were using to normalize here is called the Box-Cox Transformation. Let's take a quick peek at what normalizing some data looks like:

In [ ]:
# normalize the exponential data with boxcox
normalized_data = stats.boxcox(original_data)

# plot both together to compare
fig, ax=plt.subplots(1,2)
sns.distplot(original_data, ax=ax[0])
ax[0].set_title("Original Data")
sns.distplot(normalized_data[0], ax=ax[1])
ax[1].set_title("Normalized data")

* 기존 right-skewed된 original data에서, 처리를 통해 range의 변화가 일어날 뿐 아니라 bell-curve(=normal distribution)의 형태로 shape가 변형되었습니다.
* 이와 같이, **분포가 변형되는(shape가 변형되는)과정에서 range의 변화가 일어나는 것이 Normalization입니다.**

### **<a id='232'> 2-3-2. 정규화(Normalization) </a>**
#### **본래의 데이터로 돌아와서,**  right-skewed된 종속변수(Y)를 normalize해주기 위하여 square-root, fourth-root, log transformation을 사용하며 이를 비교하도록 하겠습니다.

In [ ]:
#histogram

fig = plt.figure(figsize = (8, 6))

fig.add_subplot(1, 3, 1)
sns.distplot(sp.sqrt(train['price'])).set_title('square-root') # square root transformation

fig.add_subplot(1, 3, 2)
sns.distplot(train['price'] ** (1/float(4.0))).set_title('fourth-root') # fourth square root transformation

fig.add_subplot(1, 3, 3)
sns.distplot(sp.special.log1p(train['price'])).set_title('log1p') # log1p transformation

#### * 종속변수(price)와 독립변수(sqft_living(주거 공간의 평방 피트(면적)))을 통한 독립변수(Y)의 Normalization 전후 비교

In [ ]:
# price에 square-root를 취함
data = pd.concat([train['price'], train['sqft_living']], axis = 1)
f, ax = plt.subplots(figsize = (8, 6))
fig = sns.regplot(x = 'sqft_living', y = 'price', data = data)

In [ ]:
# price에 fourth-root를 취함
data = pd.concat([train['price'] ** (1/float(4.0)), train['sqft_living']], axis = 1)
f, ax = plt.subplots(figsize = (8, 6))
fig = sns.regplot(x = 'sqft_living', y = 'price', data = data)

* square root보다는 fourth-root transformation을 함이 맞으나, log1p와 비교해서보자

In [ ]:
fig = plt.figure(figsize = (8, 6))

fig.add_subplot(1, 3, 1)
data = pd.concat([pd.DataFrame(sp.sqrt(train['price'])), train['sqft_living']], axis = 1)
data.columns.values[0] = 'price'
reg = sns.regplot(x = 'sqft_living', y = 'price', data = data).set_title('square-root transformation')

fig.add_subplot(1, 3, 2)
data = pd.concat([(train['price'] ** 1/float(4.0)), train['sqft_living']], axis = 1)
reg = sns.regplot(x = 'sqft_living', y = 'price', data = data).set_title('fourth-root transformation')

fig.add_subplot(1, 3, 3)
data = pd.concat([sp.special.log1p(train['price']), train['sqft_living']], axis = 1)
reg = sns.regplot(x = 'sqft_living', y = 'price', data = data).set_title('log1p transformation')

#### 이러한 비교를 통해, **독립변수에 log transformation**을 사용함이 옳다고 판단하여 log1p로 진행하고자 합니다.
* log1p는 log(1+x)로서, log 0이 발생할 경우를 대비하여 log transformation을 대신하여 practical하게 사용됩니다,

In [ ]:
train['price'] = sp.special.log1p(train['price'])

train data의 price에 log transformation을 해주었으므로, 추후 모델을 통해 예측된 최종값에 exponential을 취해 원래의 값으로 제출해야함을 잊지 말아야 합니다.

### **<a id='24'> 2-4. 독립변수(X) </a>**
* 범주형 변수의 시각화는 boxplot
* 연속형 변수의 시각화는 scatter plot을 사용하여 탐색하고자 합니다.

In [ ]:
# correlation이 높은 상위 10개의 heatmap
# continuous + sequential variables --> spearman
# abs는 반비례관계도 고려하기 위함
# https://www.kaggle.com/junoindatascience/let-s-eda-it 준호님이 수정해 준 코드로 사용하였습니다. 

cor_abs = abs(train.corr(method='spearman')) 
cor_cols = cor_abs.nlargest(n=10, columns='price').index # price과 correlation이 높은 column 10개 뽑기(내림차순)
# spearman coefficient matrix
cor = sp.array(sp.stats.spearmanr(train[cor_cols].values))[0] # 10 x 10
print(cor_cols.values)
plt.figure(figsize=(10,10))
sns.set(font_scale=1.25)
sns.heatmap(cor, fmt='.2f', annot=True, square=True , annot_kws={'size' : 8} ,xticklabels=cor_cols.values, yticklabels=cor_cols.values)

### price와 상관관계가 큰 상위 9개를 기준으로 먼저 EDA를 진행하도록 하겠습니다.

### **- grade(집의 등급)**

In [ ]:
data = pd.concat([train['price'], train['grade']], axis = 1)
f, ax = plt.subplots(figsize = (8, 6))
fig = sns.boxplot(x = 'grade', y = 'price', data = data)

* 3등급의 경우, 4등급의 Q3보다 높은 평균을 보입니다.
* 5등급 - 11등급까지 상위 이상치가 다수 존재하며, 11등급은 유독 떨어진 이상치가 존재합니다.
* 6 - 10까지는 하위 이상치 또한 존재합니다.

### ** - sqft_living(주거 공간의 평방 피트(면적)) **

In [ ]:
data = pd.concat([train['price'], train['sqft_living']], axis = 1)
f, ax = plt.subplots(figsize = (8, 6))
fig = sns.regplot(x = 'sqft_living', y = 'price', data = data)

* sqft_living이 12000인 값까지 면적 상승에 따른 가격의 상승을 따르지만, 14000인 값은 면적이 더욱 상승되었으나 가격이 12000보다 쌉니다.

### ** - sqft_living15 ** (2015년 기준 주거 공간의 평방 피트)

In [ ]:
data = pd.concat([train['price'], train['sqft_living15']], axis = 1)
f, ax = plt.subplots(figsize = (8, 6))
fig = sns.regplot(x = 'sqft_living15', y = 'price', data = data)

* 재건축으로 인해 sqft_living보다 분산이 크지만, 데이터들이 경향성을 잘 따르는 듯 합니다.

### ** - sqft_above(지하실을 제외한 평방 피트) **

In [ ]:
data = pd.concat([train['price'], train['sqft_above']], axis = 1)
f, ax = plt.subplots(figsize = (8, 6))
fig = sns.regplot(x = 'sqft_above', y = 'price', data = data)

* sqft_above의 크기에 따라 price도 높아지는 경향성을 따르고 있는 듯 보입니다. 

### ** - bathrooms(화장실의 수) **

In [ ]:
data = pd.concat([train['price'], train['bathrooms']], axis=1)
f, ax = plt.subplots(figsize=(18, 6))
fig = sns.boxplot(x='bathrooms', y="price", data=data)

bathrooms같은경우 소숫점이 있어서 많이 헷갈릴 수 있습니다. 각각의 값이 의미하는것은 아래와 같고 위의 값만을 가지고 아래의 값들이 더해져 최종 bathrooms를 이루고 있기에 각각의 값이 몇개있는지는 구분하기 힘들어 보입니다.
- 0.5 : 세면대, 화장실
- 0.75 : 세면대, 화장실, 샤워실
- 1 : 세면대, 화장실, 샤워실, 욕조

### ** - bedrooms(침실의 수)**

In [ ]:
data = pd.concat([train['price'], train['bedrooms']], axis=1)
f, ax = plt.subplots(figsize=(18, 6))
fig = sns.boxplot(x='bedrooms', y="price", data=data)

* bedrooms 변수의 값이 2 - 6에서 상위 이상치가, 1 - 3에서 하위 이상치가 다수 존재합니다.

In [ ]:
test[test['bedrooms'] > 10]

* bedrooms 변수의 특이한 점은, train 데이터는 0-10까지 존재하나 test 데이터는 11, 33값을 갖는 row가 존재한다는 점입니다.

### ** - floors **

In [ ]:
data = pd.concat([train['price'], train['floors']], axis=1)
f, ax = plt.subplots(figsize=(18, 6))
fig = sns.boxplot(x='floors', y="price", data=data)

또한, 앞선 Data Fields와 train.head()를 통해 보았던 sqft_living, sqft_lot의 값과 해당 변수들의 15년도 값의 차이를 체크해보고자 합니다.
앞에서 언급했던 내용은 다음과 같습니다.
  * sqft_living에서 sqft_living15로 가며 면적이 다소 증가함이 보입니다.
  * sqft_lot은 sqft_living과 다르게 sqft_lot15로 가며 다소 감소했음을 알 수 있습니다.

In [ ]:
train.head()

In [ ]:
train['sqft_living_gap'] = train['sqft_living15'] - train['sqft_living']
train['sqft_lot_gap'] = train['sqft_lot15'] - train['sqft_lot']

### ** - sqft_living_gap(15년 이전과 이후 주거면적의 차이) **

In [ ]:
train['sqft_living_gap'].describe()

In [ ]:
data = pd.concat([train['price'], train['sqft_living_gap']], axis = 1)
f, ax = plt.subplots(figsize = (8,6))
fig = sns.regplot(x = 'sqft_living_gap', y = 'price', data = data)

그래프를 보았을 때 감소하는 경향이 있는 것으로 보이며 그래프와 describe를 통해 보았을 때 15년도의 주거공간 평방 피트로 변화하며 증가한 값들 보다 크게 감소한 값들이 더욱 눈에 띕니다.

In [ ]:
train['sqft_lot_gap'].describe()

In [ ]:
data = pd.concat([train['price'], train['sqft_lot_gap']], axis = 1)
f, ax = plt.subplots(figsize = (8,6))
fig = sns.regplot(x = 'sqft_lot_gap', y = 'price', data = data)

데이터가 0에 매우 몰려있으며, 그래프 상 감소하는 경향을 표하고 있으나  어떠한 경향성이 있다고 파악하기는 다소 힘들어 보입니다.
해당 변수들은 파생변수로서 유지 후, 모델에서의 importance를 보도록 하겠습니다.

## **<a id='3'> 3. 데이터 전처리 </a>**

### **<a id='32'> 3-1. 독립변수(X) </a>**

### **- sqft_living(주거 공간의 평방 피트(면적))**

In [ ]:
data = pd.concat([train['price'], train['sqft_living']], axis = 1)
f, ax = plt.subplots(figsize = (8,6))
fig = sns.regplot(x = 'sqft_living', y = 'price', data = data)

In [ ]:
sorted(set(test['sqft_living']) - set(train['sqft_living']), reverse = True)[0:10]

* Test에 있으며 Train에 없는 sqft_living의 값을 체크

In [ ]:
max(test['sqft_living'])

* test 데이터 중 sqft_living의 가장 큰 값이 9640이므로, 해당 크기에 비해 다소 차이가 있는 sqft_living이 12000이상인 2가지 값을 이상치로 판단하여 제거하고자 함

In [ ]:
train.loc[train['sqft_living'] > 12000]

In [ ]:
train = train.loc[(train['id'] != 5108) & (train['id'] != 8912),]

### **- grade(집의 등급)**

In [ ]:
data = pd.concat([train['price'], train['grade']], axis = 1)
f, ax = plt.subplots(figsize = (8,6))
fig = sns.boxplot(x = 'grade', y = 'price', data = data)

In [ ]:
train[train['grade'] == 3]

* grade가 3인 2개의 row는 yr_built가 1920, 1950년으로 오래되었으며 재건축되지 않았고 sqft_lot, sqft_lot15 값이 큽니다.

In [ ]:
train.loc[(train['price']>14.5) & (train['grade'] == 7)]

* grade 7중, 가장 위 떨어진 id 12346을 이상치로 판단하여 제거하고자 합니다

In [ ]:
train.loc[(train['price']>14.5) & (train['grade'] == 8)]

* grade 8 중에서는, 가장 위에 떨어진 price가 14.8이상인 id 7173을 이상치로 판단하여 제거하고자 함

In [ ]:
train.loc[(train['price'] > 15.5) & (train['grade'] == 11)]

* grade == 11중, 가장 위 동떨어진 id 2775 또한 이상치로 판단하여 제거하고자 함

In [ ]:
train = train.loc[(train['id'] != 12346) & (train['id'] != 7173) & (train['id'] != 2775)]

### **- sqft_lot(부지 평방피트)**

In [ ]:
f, ax = plt.subplots(figsize = (8, 6))
sns.distplot(train['sqft_lot'])

sqft_lot 변수 또한 지나치게 치우친 경향이 있기에 log transformation을 취해 skewness의 완화가 필요해 보입니다.

In [ ]:
f, ax = plt.subplots(figsize = (8, 6))
sns.distplot(sp.special.log1p(train['sqft_lot']))

log transformation 이전에 비해 완화되었음을 확인할 수 있습니다.

In [ ]:
data = pd.concat([train['price'], sp.special.log1p(train['sqft_lot'])], axis = 1)
f, ax = plt.subplots(figsize = (8,6))
fig = sns.regplot(x = 'sqft_lot', y = 'price', data = data)

## 추가 정규화
코드는 해당 링크를 활용하였습니다. https://www.kaggle.com/kcs93023/2019-ml-month-2nd-baseline

또한, 코드를 추가 및 수정하며 Data Fields에서 말씀드린 점을 파생변수로 만들어서 체크해보고자 하였습니다.

파생변수중 일부는 https://www.kaggle.com/marchen911/xgboost-lightgbm-catboost 를 참조하였습니다.

변수 수정

In [ ]:
for df in [train,test]:
    df['year'] = df.date.apply(lambda x: x[0:4]).astype(int)
    df['month'] = df.date.apply(lambda x: x[4:6]).astype(int)
    df['day'] = df.date.apply(lambda x: x[6:8]).astype(int)
    df['date'] = df['date'].apply(lambda x: x[0:8])
    df['yr_renovated'] = df['yr_renovated'].apply(lambda x: sp.nan if x == 0 else x)
    df['yr_renovated'] = df['yr_renovated'].fillna(df['yr_built'])

#### 변수 생성

In [ ]:
for df in [train,test]:
    # bedrooms와 bathrooms의 개수를 종합한 방의 개수
    df['total_rooms'] = df['bedrooms'] + df['bathrooms']
    
    # grade와 condition의 곱을 통한 새로운 지표 생성
    df['grade_condition'] = df['grade'] * df['condition']
    
    # 면적관련 변수들을 이용한 파생변수 생성
    
    # 15년 변화 이전 부지와 주거공간의 평방피트 합
    df['sqft_total'] = df['sqft_living'] + df['sqft_lot'] 
    
    # 15년 변화 이전 부지, 주거공간, 지하실 제외, 지하실의 평방피트 합
    df['sqft_total_size'] = df['sqft_living'] + df['sqft_lot'] + df['sqft_above'] + df['sqft_basement']
    
    # 15년 변화 이후 부지와 주거공간의 평방피트 합
    df['sqft_total15'] = df['sqft_living15'] + df['sqft_lot15'] 
    
    # 주거공간 평방피트의 변화 이전과 변화 이후의 차이
    df['sqft_living_gap'] = sp.absolute(df['sqft_living15'] - df['sqft_living'])
    
    # 15년 변화 이후 부지 평방피트의 차이
    df['sqft_lot_gap'] = sp.absolute(df['sqft_lot15'] - df['sqft_lot'])
    
    # 재건축 여부를 통한 파생변수 생성
    df['is_renovated'] = df['yr_renovated'] - df['yr_built']
    df['is_renovated'] = df['is_renovated'].apply(lambda x: 0 if x == 0 else 1)
    df['date'] = df['date'].astype('int')
    
    df['garret'] = (df.floors%1==0.5).astype(int)
    df['diff_of_rooms'] = sp.absolute(df['bedrooms'] - df['bathrooms'])
    df['living_per_floors'] = df['sqft_living'] / df['floors']
    df['total_score'] = df['condition'] + df['grade'] + df['view']
    df['living_per_lot'] = df['sqft_living'] / df['sqft_lot']
    df['gap_living_per_floor'] = df['sqft_living_gap'] / df['floors']
    df['exist_special'] = df.garret + df.waterfront + df.is_renovated

In [ ]:
train['per_price'] = train['price']/train['sqft_total_size']
zipcode_price = train.groupby(['zipcode'])['per_price'].agg({'mean','var'}).reset_index()
train = pd.merge(train,zipcode_price,how='left',on='zipcode')
test = pd.merge(test,zipcode_price,how='left',on='zipcode')

for df in [train,test]:
    df['zipcode_mean'] = df['mean'] * df['sqft_total_size']
    df['zipcode_var'] = df['var'] * df['sqft_total_size']
    del df['mean']; del df['var']
    
del train['per_price']

In [ ]:
# transformation
skew_columns = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']

for c in skew_columns:
    train[c] = sp.special.log1p(train[c].values)
    test[c] = sp.special.log1p(test[c].values)

## **<a id='5'> 5. 모델링 </a>**

#### **<a id='51'>5-1. Linear Regression</a>**

In [ ]:
train.head()

* Linear Regression을 사용할 경우, 범주형 변수(categorical variable)에 대해서 더미변수(dummy variable)로 변환하여야 합니다. 
* bedrooms, bathrooms, floors, view, condition, grade, total_rooms, grade_condition, exist_special, living_per_lot, living_per_floors, gap_living_per_floor, total_score, diff_of_rooms 변수들은 데이터 값의 크기에 따른 영향이 있다고 판단하여 label encoder를 적용해주도록 하겠습니다.
* waterfront, zipcode, yr_built, yr_renovated, garret, is_renovated 변수들에 대하여 pandas 라이브러리 내 get_dummies 함수를 이용하여 더미변수화 하도록 하겠습니다.
* date의 경우, year, month, day로 더미변수를 생성하였으므로 이를 이용해 년, 월, 일로 나누어 더미변수로 변환 후 모델 성능을 비교해보려 합니다.

In [ ]:
X_features_dummy = pd.get_dummies(train, columns = ['waterfront', 'zipcode', 'yr_built', 'yr_renovated', 'year', 'month', 'day', 'garret', 'is_renovated'])
Y_features_dummy = pd.get_dummies(test, columns = ['waterfront', 'zipcode', 'yr_built', 'yr_renovated', 'year', 'month', 'day', 'garret', 'is_renovated'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

le_columns = ['bedrooms', 'bathrooms', 'floors', 'view', 'condition', 'grade', 'total_rooms', 'grade_condition', 'exist_special', 'living_per_lot', 'living_per_floors',
              'gap_living_per_floor', 'total_score', 'diff_of_rooms']


le = LabelEncoder()

for i in le_columns : 
    X_features_dummy[i] = le.fit_transform(X_features_dummy[i])
    Y_features_dummy[i] = le.fit_transform(Y_features_dummy[i])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

test_id = Y_features_dummy['id']
Y_test = Y_features_dummy.drop(['id'], axis = 1, inplace = False)
y_target = X_features_dummy['price']
X_data = X_features_dummy.drop(['price', 'id'], axis = 1, inplace = False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size = 0.25, random_state = 42)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
pred = sp.special.expm1(lr.predict(X_test))
y_test = sp.special.expm1(y_test)
mse = mean_squared_error(y_test, pred)
rmse = mse ** float(0.5)
print('RMSE : {0:.3F}'.format(rmse))

* 시각화 및 설명의 용이성을 위해 OLS(Ordinary Least Square)를 통해 보고자 합니다.

In [ ]:
import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

y = X_features_dummy['price']
X = X_features_dummy.drop(['price', 'id'], axis = 1, inplace = False)
tests = Y_features_dummy

# X_features_dummy 데이터 내, 모든 X를 이용하여 Y인 price를 예측하도록 함
result = sm.OLS(y, X).fit()

print(result.summary())

다중공선성(multicollinearity problem)이 존재함을 Warnings에서 말해주고 있습니다.

선형회귀를 사용할 경우, 기본적인 4가지의 기본 가정이 있습니다.

>   1. **선형성** 
        * 예측하고자 하는 y와 독립변수 x간의 선형성 만족
   
>   2. **독립성**
        * 독립변수 x간 서로 독립성을 만족
    
>   3. **등분산성**
    
>   4. **정규성**
        * 잔차의 정규성 만족

multicollinearity의 존재유무로 인해, 독립성의 가정이 지켜지지 않음을 알 수 있습니다. 또한, 이러한 가정에 대해 자세히 보고자 합니다.

In [ ]:
# fitted values
model_fitted_y = result.fittedvalues

# model residuals
model_residuals = result.resid

# normalized residuals
model_norm_residuals = result.get_influence().resid_studentized_internal

# absolute squared normalized residuals
model_norm_residuals_abs_sqrt = sp.sqrt(sp.absolute(model_norm_residuals))

# absolute residuals
model_abs_resid = sp.absolute(model_residuals)

# leverage, from statsmodel internals
model_leverage = result.get_influence().hat_matrix_diag

# cook's distance, from sttasmodels internals
model_cooks = result.get_influence().cooks_distance[0]

## **Residual Plot**

In [ ]:
plot_lm_1 = plt.figure(1)
plot_lm_1.set_figheight(8)
plot_lm_1.set_figwidth(12)

plot_lm_1.axes[0] = sns.residplot(model_fitted_y, 'price', data=X_features_dummy, 
                          lowess=True, 
                          scatter_kws={'alpha': 0.5}, 
                          line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_1.axes[0].set_title('Residuals vs Fitted')
plot_lm_1.axes[0].set_xlabel('Fitted values')
plot_lm_1.axes[0].set_ylabel('Residuals')

# annotations
abs_resid = model_abs_resid.sort_values(ascending=False)
abs_resid_top_3 = abs_resid[:3]

for i in abs_resid_top_3.index:
    plot_lm_1.axes[0].annotate(i, 
                               xy=(model_fitted_y[i], 
                                   model_residuals[i]));

Residuals vs Fitted Plot은 x축에 $\hat{Y}$, y축에 잔차 ${\epsilon }_{i}$로 출력합니다. 선형회귀에서 오차는 평균이 0이고 분산이 일정한 정규분포를 가정하기에 $\hat{Y}$과 무관하게 잔차의 평균은 0을 중심으로 일정하게 패턴 없이 분포되어 있어야 합니다.

## **QQ Plot**

In [ ]:
from statsmodels.graphics.gofplots import ProbPlot

QQ = ProbPlot(model_norm_residuals)
plot_lm_2 = QQ.qqplot(line='45', alpha=0.5, color='#4C72B0', lw=1)

plot_lm_2.set_figheight(8)
plot_lm_2.set_figwidth(12)

plot_lm_2.axes[0].set_title('Normal Q-Q')
plot_lm_2.axes[0].set_xlabel('Theoretical Quantiles')
plot_lm_2.axes[0].set_ylabel('Standardized Residuals');

# annotations
abs_norm_resid = sp.flip(sp.argsort(sp.absolute(model_norm_residuals)), 0)
abs_norm_resid_top_3 = abs_norm_resid[:3]

for r, i in enumerate(abs_norm_resid_top_3):
    plot_lm_2.axes[0].annotate(i, 
                               xy=(sp.flip(QQ.theoretical_quantiles, 0)[r],
                                   model_norm_residuals[i]));

잔차의 정규성이 이루어지지 않음을 확인할 수 있습니다. model의 normalized residual값이 -2 이하부터 정규성을 갖추지 못하고 있음을 알 수 있습니다.

## **Scale-Location Plot**

In [ ]:
plot_lm_3 = plt.figure(3)
plot_lm_3.set_figheight(8)
plot_lm_3.set_figwidth(12)

plt.scatter(model_fitted_y, model_norm_residuals_abs_sqrt, alpha=0.5)
sns.regplot(model_fitted_y, model_norm_residuals_abs_sqrt, 
            scatter=False, 
            ci=False, 
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_3.axes[0].set_title('Scale-Location')
plot_lm_3.axes[0].set_xlabel('Fitted values')
plot_lm_3.axes[0].set_ylabel('$\sqrt{|Standardized Residuals|}$');

# annotations
abs_sq_norm_resid = sp.flip(sp.argsort(model_norm_residuals_abs_sqrt), 0)
abs_sq_norm_resid_top_3 = abs_sq_norm_resid[:3]

for i in abs_norm_resid_top_3:
    plot_lm_3.axes[0].annotate(i, 
                               xy=(model_fitted_y[i], 
                                   model_norm_residuals_abs_sqrt[i]));


Scale-location plot은 x축에 $\hat{Y}$, y축에는 표준화 잔차(Standard Residuals)를 보입니다. 빨간 추세선이 가장 이상적이며, 벗어난 값들은 이상치(outlier)일 가능성이 있습니다.

## **Leverage Plot**

In [ ]:
plot_lm_4 = plt.figure(4)
plot_lm_4.set_figheight(8)
plot_lm_4.set_figwidth(12)

plt.scatter(model_leverage, model_norm_residuals, alpha=0.5)
sns.regplot(model_leverage, model_norm_residuals, 
            scatter=False, 
            ci=False, 
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_4.axes[0].set_xlim(0, 0.20)
plot_lm_4.axes[0].set_ylim(-3, 5)
plot_lm_4.axes[0].set_title('Residuals vs Leverage')
plot_lm_4.axes[0].set_xlabel('Leverage')
plot_lm_4.axes[0].set_ylabel('Standardized Residuals')

# annotations
leverage_top_3 = sp.flip(sp.argsort(model_cooks), 0)[:3]

for i in leverage_top_3:
    plot_lm_4.axes[0].annotate(i, 
                               xy=(model_leverage[i], 
                                   model_norm_residuals[i]))
    
# shenanigans for cook's distance contours
def graph(formula, x_range, label=None):
    x = x_range
    y = formula(x)
    plt.plot(x, y, label=label, lw=1, ls='--', color='red')

p = len(result.params) # number of model parameters

graph(lambda x: sp.sqrt((0.5 * p * (1 - x)) / x), 
      sp.linspace(0.001, 0.200, 50), 
      'Cook\'s distance') # 0.5 line
graph(lambda x: sp.sqrt((1 * p * (1 - x)) / x), 
      sp.linspace(0.001, 0.200, 50)) # 1 line
plt.legend(loc='upper right');

Leverage는 설명변수가 얼마나 극단에 치우쳐 있는지 말합니다.

### **위에서 본 바와 같이**
선형회귀는 기본 4가지 가정이 모두 충족되어야 하기에 독립성 가정부터 만족되어야 합니다. 이를 위해, 다중공선성을 제거하고자 사용할 수 있는 방법은 (1) VIF 값을 기준으로 공선성을 야기시키는 변수를 제거, (2) 데이터의 row를 늘려줌으로써 표본의 크기를 증가, (3) 변수의 변환 등이 있습니다. 

하지만, competition 내에서 데이터의 row를 늘려줌은 불가능하며 VIF값을 기준으로 변수들을 제거해 나아가는 점은 기존 변수들을 이용해 생성한 파생변수를 사용하지 못하게 될 문제들이 있습니다. 그렇기에, 다중공선성에 robust하며 평균적으로 성능이 좋은 Tree 기반 모델을 사용합니다.

저 또한, 이후 Tree 기반 모델들을 사용하여 예측해보고자 합니다.


#### **<a id='52'> 5-2. Random Forest </a>**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [ ]:
test_id = test['id']
Y_test = test.drop(['id'], axis = 1, inplace = False)

In [ ]:
y_target = train['price']
X_data = train.drop(['price', 'id'], axis = 1, inplace = False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size = 0.25, random_state = 42)

In [ ]:
forest_reg = RandomForestRegressor()

In [ ]:
import numpy as np
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
param_dist = {"max_depth": [7, 11, 15, 18, 21],
              "max_features": sp_randint(1, 31),
              "min_samples_split": sp_randint(2, 21),
              "min_samples_leaf": sp_randint(1, 21),
              "bootstrap": [True, False],
              "random_state": [42]
             }

n_iter_search = 20
random_search = RandomizedSearchCV(forest_reg, param_distributions=param_dist,
                                   n_iter=n_iter_search)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
report(random_search.cv_results_)

randomizedsearch를 통해 높은 score를 보이는 parameter를 파악하였습니다.

In [ ]:
from sklearn.metrics import mean_squared_error as mse

In [ ]:
pred = sp.special.expm1(random_search.predict(X_test))
y_test = sp.expm1(y_test)
rmse = (mean_squared_error(y_test, pred)) ** float(0.5)
print('RMSE : {0:.3F}'.format(rmse))

In [ ]:
test_pred = sp.special.expm1(random_search.predict(Y_test))

In [ ]:
submission = pd.DataFrame({'id' : test_id, 'price' : test_pred})

In [ ]:
submission.to_csv('rfgridcv.csv', index=False)


#### **<a id='53'> 5-3. XGBoost </a>**


XGBoost의 설명은 다음 article을 참조하였습니다. 

https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

-------------------------------------------------

### **XGBoost Advantage**

1. **Regularization** : Standard GBM(Gradient Boosting Machine)은 overfitting을 줄이는 데 도움이 됩니다.

2. **Parallel Processing** : XGBoost는 병렬 처리를 구현하며 GBM과 비교해 매우 빠릅니다.

3. **High Flexibility** : XGBoost는 유저가 custom optimization objectives와 평가 지표(evaluation criteria)를 정의할 수 있습니다.

4. **Handling Missing Values** : XGBoost는 missing value를 핸들링하는 routine이 in-built되어 있습니다.

5. **Tree Pruning** : GBM은 greedy algorithm과 같이, negative loss가 발생하면 node split을 중단함에 반해, XGBoost는 지정된 max_depth까지 split하고 트리를 거꾸로 잘라 positive gain이 없으면 pruning을 합니다.

6. **Built-in Cross-Validation** : XGBoost는 유저가 매 iteration마다 cross-validation을 실행할 수 있도록 합니다.

7. **Continue on Existing Model** : 이전에 실행시킨 last iteration으로부터 XGBoost model을 training시킬 수 있습니다.


* XGBoost는 해당 커널을 참조하여 작성토록 했음을 명시합니다. https://www.kaggle.com/marchen911/xgboost-lightgbm-catboost

In [ ]:
from xgboost import XGBRegressor
from sklearn import model_selection

In [ ]:
test_id = test['id']
Y_test = test.drop(['id'], axis = 1, inplace = False)

In [ ]:
y_target = train['price']
X_data = train.drop(['price', 'id'], axis = 1, inplace = False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size = 0.25, random_state = 42)
watchlist=[(X_train,y_train),(X_test,y_test)]

In [ ]:
# feature missmatch 에러를 해결하기 위함
Y_test = Y_test[X_test.columns]

In [ ]:
xgb_model= XGBRegressor(tree_method='gpu_hist',
                        n_estimators=100000,
                        num_round_boost=500,
                        show_stdv=False,
                        feature_selector='greedy',
                        verbosity=0,
                        reg_lambda=10,
                        reg_alpha=0.01,
                        learning_rate=0.001,
                        seed=42,
                        colsample_bytree=0.8,
                        colsample_bylevel=0.8,
                        subsample=0.8,
                        n_jobs=-1,
                        gamma=0.005,
                        base_score=np.mean(y_target)
                        )

In [ ]:
xgb_model.fit(X_train,y_train, verbose=False, eval_set=watchlist,
              eval_metric='rmse',
              early_stopping_rounds=1000)

In [ ]:
xgb_score=mse(sp.special.expm1(xgb_model.predict(X_test)),sp.special.expm1(y_test))**0.5

print("RMSE : {}".format(xgb_score))

In [ ]:
xgb_pred = sp.special.expm1(xgb_model.predict(Y_test))

In [ ]:
submission=pd.read_csv('../input/sample_submission.csv')
submission['price']= xgb_pred
submission.to_csv('xgb_submission.csv', index = False)


#### **<a id='54'> 5-4. LightGBM </a>**

lightGBM모델은 해당 커널을 참조하여 작성하였음을 명시합니다. https://www.kaggle.com/marchen911/xgboost-lightgbm-catboost

In [ ]:
import lightgbm as lgb

In [ ]:
y_target = train['price']
X_data = train.drop(['price', 'id'], axis = 1, inplace = False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size = 0.2, random_state = 42)

model_lgb=lgb.LGBMRegressor(
                           learning_rate=0.001,
                           n_estimators=100000,
                           subsample=0.6,
                           colsample_bytree=0.6,
                           reg_alpha=0.2,
                           reg_lambda=10,
                           num_leaves=35,
                           silent=True,
                           min_child_samples=10,
                            
                           )

model_lgb.fit(X_train,y_train,eval_set=(X_test,y_test),verbose=0,early_stopping_rounds=1000,
              eval_metric='rmse')

lgb_score=mse(sp.special.expm1(model_lgb.predict(X_test)),sp.special.expm1(y_test))**0.5
print("RMSE unseen : {}".format(lgb_score))

In [ ]:
lgb_pred = sp.special.expm1(model_lgb.predict(Y_test))

In [ ]:
submission=pd.read_csv('../input/sample_submission.csv')
submission['price']= lgb_pred
submission.to_csv('lgbm_submission.csv', index = False)

**Ensemble**

In [ ]:
score=lgb_score+xgb_score
lgb_ratio=1-lgb_score/score
xgb_ratio=1-xgb_score/score
predict=lgb_pred*(lgb_ratio)+xgb_pred*(xgb_ratio)
print('lgb_ratio={}, xgb_ratio={}'.format(lgb_ratio,xgb_ratio))
submission=pd.read_csv('../input/sample_submission.csv')
submission.loc[:,'price']=predict
submission.to_csv('xgb_lgbm.csv',index=False)